In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import sys
sys.path.append('/notebooks/carputils')
sys.path.append('/notebooks/examples')

sys.path

['',
 '/opt/conda/lib/python27.zip',
 '/opt/conda/lib/python2.7',
 '/opt/conda/lib/python2.7/plat-linux2',
 '/opt/conda/lib/python2.7/lib-tk',
 '/opt/conda/lib/python2.7/lib-old',
 '/opt/conda/lib/python2.7/lib-dynload',
 '/opt/conda/lib/python2.7/site-packages',
 '/opt/conda/lib/python2.7/site-packages/Sphinx-1.5.1-py2.7.egg',
 '/opt/conda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/notebooks/carputils',
 '/notebooks/examples']

In [3]:
cat ../examples/tutorials/02_EP_tissue/01_basic_usage/run.py

#!/usr/bin/env python

r"""
Overview

This tutorial introduces to the basics of using the carpentry executable 
for simulating EP at the tissue and organ scale.
Details on how to inquire input parameters are provided in the :ref:`carpentry <carpentry>` section
of the manual.

In absence of pacemaker currents electrical stimulus currents must be supplied in order to 
depolarize tissue above the threshold to elicit a propagated action potential. 
The simplest way of initiating a propagating action potential is the use of a transmembrane stimulus, :math:`I_{\mathrm{tr}}`.
According to the definition in CARPentry supplying a positive :math:`I_{\mathrm{tr}}` depolarizes the tissue,
a negative :math:`I_{\mathrm{tr}}` hyperpolarizes the tissue.


Experimental setup

To introduce definition and usage of transmembrane stimulus currents in CARPentry a simple experiment 
using a thin monolayer preparation was defined. The following input parameters are available to steer the experiment:

.. code-

In [4]:
%run -i ../examples/tutorials/02_EP_tissue/01_basic_usage/run.py --duration 20 --S1-strength 20. --S1-dur 15



Could not find environment variable $CARPUTILS_SETTINGS
The env variable should point to path-to-carputils/settings.yaml!

SETTINGS SUMMARY
Executables
----------------------------------------------------------------------
   CARP:            /notebooks/carputils/bin/carphelp
   MESHER:          /notebooks/carputils/bin/mesher
   FSM:             < NOT FOUND >
   BENCH:           < NOT FOUND >
   GLINTERPOLATE:   < NOT FOUND >
   GLGRADIENT:      < NOT FOUND >
   GLELEMCENTERS:   < NOT FOUND >
   GLMESHCONVERT:   < NOT FOUND >
   GLVTKCONVERT:    < NOT FOUND >
   GLTRANSFER:      < NOT FOUND >
   GLRULEFIBERS:    < NOT FOUND >
   ELLIPTICSOLVER:  < NOT FOUND >
   PARABOLICSOLVER: < NOT FOUND >
   ADJOINTSOLVER:   < NOT FOUND >
   GLFILAMENT:      < NOT FOUND >
   GLIGBPROCESS:    < NOT FOUND >
   SURFACEMESH:     < NOT FOUND >
   GLSURFACECONVERT: < NOT FOUND >
   MESHTOOL:        < NOT FOUND >
   ELASTICITY:      < NOT FOUND >
   FLUIDSOLVE:      < NOT FOUND >
   CVSTOOL:         < 

/notebooks/carputils/carputils/settings/buildinfo.py:81: UserWarning: Error retrieving -buildinfo from CARP - assuming no mechanics or CUDA
  warn('Error retrieving -buildinfo from CARP - assuming no mechanics '


In [9]:
import carputils.cml

In [ ]:
import os
from datetime import date

from carputils import settings
from carputils import tools
from carputils import mesh
from carputils import testing

def parser():
    parser = tools.standard_parser()
    group  = parser.add_argument_group('experiment specific options')
    group.add_argument('--duration',
                        type=float, default=20.,
                        help='Duration of simulation (ms)')
    group.add_argument('--S1-strength',
                         type=float, default=20.,
                        help='pick transmembrane current stimulus strength in [uA/cm^2] (default is 20.)') 
    group.add_argument('--S1-dur',
                        type=float, default=2.,
                        help='pick transmembrane current stimulus duration in [ms] (default is 2.)') 
    return parser

def jobID(args):
    """
    Generate name of top level output directory.
    """
    today = date.today()
    return '{}_basic_{}'.format(today.isoformat(), args.duration)

@tools.carpexample(parser, jobID)
def run(args, job):

    # Generate mesh
    # Units are mm
    
    # Block which is thin in z direction
    geom = mesh.Block(size=(10., 1., 0.2))
    
    # Set fibre angle to 0, sheet angle to 0
    geom.set_fibres(0, 0, 90, 90)

    # Generate and return base name
    meshname = mesh.generate(geom)

    # Define the geometry of the stimulus at one end of the block
    # Units are um

    stim = [ '-num_stim',                           1, 
             '-stimulus[0].name',                 'S1',
             '-stimulus[0].stimtype',               0,
             '-stimulus[0].strength',   args.S1_strength,
             '-stimulus[0].duration',   args.S1_dur ]

    electrode = mesh.block_boundary_condition(geom, 'stimulus', 0, 'x', True)



    # Get basic command line, including solver options
    cmd = tools.carp_cmd('basic.par')

    cmd += ['-simID',    job.ID,
            '-meshname', meshname,
            '-dt',       25,
            '-tend',     args.duration]
    cmd += stim + electrode

    if args.visualize:
        cmd += ['-gridout_i', 3]
        cmd += ['-gridout_e', 3]
        cmd += ['-spacedt', 0.1]

    # Run simulation 
    job.carp(cmd)

    # Do visualization
    if args.visualize and not settings.platform.BATCH:

        # Prepare file paths
        geom = os.path.join(job.ID, os.path.basename(meshname)+'_i')
        data = os.path.join(job.ID, 'vm.igb.gz')
        view = 'view_vm.mshz'
        
        # Call meshalyzer
        job.meshalyzer(geom, data, view)

if __name__ == '__main__':
    run()    